# User example

This notebook is an example of how to use the codes to compute the expected value of marginal distributions.

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from scipy.stats import poisson
import neuralnetwork
import convert_csv
import generate_data
import simulation
import grads

## SSA simulations of a CRN

Chosen Chemical Reaction Network: $ø \rightarrow S_1$. The reaction propensity is given by $\lambda(x) = cx$ where c is a constant.

We want to generate train, validation and test data. Because this step takes time, we save the data in csv files. 

In [ ]:
# file where propensity functions are defined
# needs to be imported because of the multiprocessing module
import propensities

In [ ]:
CRN_NAME = 'ø_S1'
datasets = {'train': 4*900, 'valid': 4*80, 'test': 4*44}
DATA_LENGTH = sum(datasets.values())
T_SAMPLES = np.array([1, 5, 10, 15])
N_TRAJ = 10**4

In [ ]:
# because we use multiprocessing
if __name__ == '__main__':

    stoich_mat = np.array([1]).reshape(1,1)
    crn = simulation.CRN(stoichiometric_mat=stoich_mat, propensities=np.array([propensities.lambda1]), n_params=1)
    dataset = generate_data.CRN_Dataset(crn=crn, sampling_times=T_SAMPLES)
    X, y = dataset.generate_data(data_length=DATA_LENGTH, n_trajectories=N_TRAJ)

    # writing CSV files
    somme = 0
    max_value = 0
    for key, value in datasets.items():
        convert_csv.array_to_csv(X[somme:somme+value,:], f'X_{CRN_NAME}_{key}')
        convert_csv.array_to_csv(y[somme:somme+value,:], f'y_{CRN_NAME}_{key}')
        somme += value

## Creating a Neural Network

In [ ]:
N_COMPS = 5
NUM_PARAMS = 1

In [ ]:
model = neuralnetwork.NeuralNetwork(n_comps=N_COMPS, n_params=NUM_PARAMS)

## Training Neural Network

In [ ]:
# loading data
X_train = convert_csv.csv_to_tensor(f'birth_data/X_{CRN_NAME}_train.csv')
y_train = convert_csv.csv_to_tensor(f'birth_data/y_{CRN_NAME}_train.csv')
X_valid = convert_csv.csv_to_tensor(f'birth_data/X_{CRN_NAME}_valid.csv')
y_valid = convert_csv.csv_to_tensor(f'birth_data/y_{CRN_NAME}_valid.csv')
X_test = convert_csv.csv_to_tensor(f'birth_data/X_{CRN_NAME}_test.csv')
y_test = convert_csv.csv_to_tensor(f'birth_data/y_{CRN_NAME}_test.csv')

train_data = [X_train, y_train]
valid_data = [X_valid, y_valid]

In [ ]:
CRN_NAME = 'ø_S1'

# loading data
X_train = convert_csv.csv_to_tensor(f'birth_data/X_{CRN_NAME}_train.csv')
y_train = convert_csv.csv_to_tensor(f'birth_data/y_{CRN_NAME}_train.csv')
X_valid = convert_csv.csv_to_tensor(f'birth_data/X_{CRN_NAME}_valid.csv')
y_valid = convert_csv.csv_to_tensor(f'birth_data/y_{CRN_NAME}_valid.csv')
X_test = convert_csv.csv_to_tensor(f'birth_data/X_{CRN_NAME}_test.csv')
y_test = convert_csv.csv_to_tensor(f'birth_data/y_{CRN_NAME}_test.csv')

train_data = [X_train, y_train[:, 1:]]
valid_data = [X_valid, y_valid[:, 1:]]

In [ ]:
neuralnetwork.train_NN(model, train_data, valid_data, loss=neuralnetwork.loss_kldivergence, max_rounds=600, lr=0.01, batchsize=64)

## Visualisation of predicted distributions

In [ ]:
def plot_model(to_pred, expectations, model, n_comps=N_COMPS, up_bound=1_100, on_same_fig = False):
    # predictions
    layer_ww, layer_rr, layer_pp = model.forward(to_pred)
    x = torch.arange(1, up_bound+1).repeat(1, n_comps,1).permute([2,0,1])
    y_pred = neuralnetwork.mix_nbpdf(layer_rr, layer_pp, layer_ww, x)
    y_pred = y_pred.detach().numpy()
    y_exp = expectations.detach().numpy()
    # plots
    if on_same_fig:
        plt.plot(np.arange(len(y_pred-1)), y_pred[:, 1:], color='blue', label='prediction')
        plt.plot(np.arange(len(y_exp)), y_exp, color='red', label='expectation')
        plt.plot(np.arange(len(y_pred)), poisson.pmf(np.arange(len(y_pred)), to_pred[0]*to_pred[1]), color='green', label='true')
        plt.legend()
        plt.show()
    else:
        fig, axs = plt.subplots(2)
        axs[0].plot(np.arange(len(y_pred)), y_pred)
        axs[0].set_title('Predictions')
        axs[1].plot(np.arange(len(y_exp)), y_exp)
        axs[1].set_title('Expectations', y=-0.4)
        plt.show()

In [ ]:
# play with the chosen index to check for different distributions
index = 0

In [ ]:
plot_model(X_test[index,:], y_test[index, 1:], model, up_bound = len(y_test[index, :]), on_same_fig=True)

### Case of a Poisson distribution

In [ ]:
from scipy.stats import poisson

In [ ]:
def plot_Poisson_model(to_pred, model, n_comps=N_COMPS, up_bound=100, on_same_fig = False):
    # predictions
    layer_ww, layer_rr, layer_pp = model.forward(to_pred)
    x = torch.arange(1, up_bound+1).repeat(1, n_comps,1).permute([2,0,1])
    y_pred = neuralnetwork.mix_nbpdf(layer_rr, layer_pp, layer_ww, x)
    y_pred = y_pred.detach().numpy()
    y_exp = poisson.pmf(np.arange(up_bound), to_pred[0]*to_pred[1])
    # plots
    if on_same_fig:
        plt.plot(np.arange(len(y_pred)), y_pred, color='blue', label='prediction')
        plt.plot(np.arange(len(y_exp)), y_exp, color='red', label='expectation')
        plt.legend()
        plt.show()
    else:
        fig, axs = plt.subplots(2)
        axs[0].plot(np.arange(len(y_pred)), y_pred)
        axs[0].set_title('Predictions')
        axs[1].plot(np.arange(len(y_exp)), y_exp)
        axs[1].set_title('Expectations', y=-0.4)
        plt.show()

In [ ]:
index=10
plot_Poisson_model(X_test[index,:], model, on_same_fig=True)

### Case of a Negative Binomial Distribution

In [ ]:
from scipy.stats import nbinom

In [ ]:
def plot_NB_model(to_pred, model, n_comps=N_COMPS, initial_state=10, up_bound=1_100, on_same_fig = False):
    # predictions
    layer_ww, layer_rr, layer_pp = model.forward(to_pred)
    x = torch.arange(1, up_bound+1).repeat(1, n_comps,1).permute([2,0,1])
    y_pred = neuralnetwork.mix_nbpdf(layer_rr, layer_pp, layer_ww, x)
    y_pred = y_pred.detach().numpy()
    y_exp = nbinom.pmf(np.arange(up_bound), initial_state, np.exp(-to_pred[0]*to_pred[1]))
    # plots
    if on_same_fig:
        plt.plot(np.arange(len(y_pred)), y_pred, color='blue', label='prediction')
        plt.plot(np.arange(len(y_exp)), y_exp, color='red', label='expectation')
        plt.legend()
        plt.show()
    else:
        fig, axs = plt.subplots(2)
        axs[0].plot(np.arange(len(y_pred)), y_pred)
        axs[0].set_title('Predictions')
        axs[1].plot(np.arange(len(y_exp)), y_exp)
        axs[1].set_title('Expectations', y=-0.4)
        plt.show()

## Compute gradients from predicted distributions

In [ ]:
# choose the input
input = X_test[0,:]
len_output = len(y_test[0, 1:])

In [ ]:
gradients = grads.grads(input, model, length_output=len_output)

In [ ]:
# plot a gradient
plt.plot(gradients)
plt.show()

### Case of a Poisson distribution

In [ ]:
def grad_lambda(t, lambd, k):
    prod = t*lambd
    return poisson.pmf(k-1, prod)/k*t*(k-prod)

In [ ]:
x = np.linspace(0, 100, 200)
y = [grad_lambda(X_test[index, 0], X_test[index, 1], k=i) for i in x]
plt.plot(grads.grads(X_test[index,:], model, length_output=100))
plt.plot(x, y, color='green')
plt.show()

## Compute expected value of gradients from predicted distributions

In [ ]:
exp_val = grads.expected_val(input, model, length_output = len_output)